In [1]:
from chargecraft.storage.storage import MoleculePropRecord, MoleculePropStore
from openff.toolkit.topology import Molecule
from openff.units import unit
from openff.recharge.esp import ESPSettings
from openff.recharge.grids import LatticeGridSettings
from openff.recharge.esp.storage import MoleculeESPRecord
from openff.recharge.charges.resp import generate_resp_charge_parameter
from openff.recharge.charges.resp.solvers import IterativeSolver
import numpy as np

resp_solver = IterativeSolver()


def calculate_resp_charges(openff_mol: Molecule,
                           grid: unit.Quantity,
                           esp: unit.Quantity,
                           qc_data_settings: ESPSettings) -> list[float]:
    """Calculate resp charges given a set of input data
    
    Parameters
    ----------
    grid: unit.Quantity
        grid in which the ESP was calculate don
    esp: unit.Quantity
        esp calculated from the qm calculation
    qc_data_settings: ESPSettings
        esp settings the esp was calculated at
        
    Returns
    -------
    list[float]
        list of resp charges
    
    
    """
    qc_data_record = MoleculeESPRecord.from_molecule(
        openff_mol, openff_mol.conformers[0], grid, esp, None, qc_data_settings
    )

    resp_charge_parameter = generate_resp_charge_parameter(
        [qc_data_record], resp_solver
    )
    
    matchs = openff_mol.chemical_environment_matches(query=resp_charge_parameter.smiles)
    resp_charges = [0.0 for _ in range(openff_mol.n_atoms)]
    for match in matchs:
        for i, atom_indx in enumerate(match):
            resp_charges[atom_indx] = resp_charge_parameter.value[i]
    
    return np.round(resp_charges, 4).tolist()

In [2]:
prop_store = MoleculePropStore("/mnt/storage/nobackup/nca121/paper_charge_comparisons/async_chargecraft_more_workers/conformer_test/qc_archive_run/conformers.db", cache_size=1000)


In [3]:
smiles_list = prop_store.list()
smiles_list[0]

'O=[N+]([O-])OCC[NH+](CCO[N+](=O)[O-])CCO[N+](=O)[O-]'

In [4]:
retrieved = prop_store.retrieve(smiles_list[0])

In [5]:
retrieved = prop_store.retrieve(smiles_list[0])[0]
tagged_smiles = retrieved.tagged_smiles
conformer = retrieved.conformer_quantity

openff_mol = Molecule.from_mapped_smiles(tagged_smiles)
openff_mol.add_conformer(conformer)
grid = retrieved.grid_coordinates_quantity#.flatten()
esp = retrieved.esp_quantity.reshape(-1,1)


In [30]:
grid

Magnitude,[[-6.363961030844296 -4.596194077712559 0.1758364724366695] [-6.363961030844296 -4.596194077712559 0.8829432536232169] [-6.363961030844296 -4.242640687119285 -0.17771691815660423] ... [7.42462120229338 -1.7677669529663684 -0.17771691815660423] [7.42462120229338 -1.7677669529663684 0.5293898630299432] [7.42462120229338 -1.4142135623730947 0.1758364724366695]]
Units,angstrom


In [35]:
esp.reshape(-1,1).shape

(4312, 1)

In [39]:
esp

Magnitude,[[0.06644077686607182] [0.06581642791264565] [0.06937766522762968] ... [0.060589985218608464] [0.060853948049358515] [0.061394512473341385]]
Units,hartree/elementary_charge


In [31]:
openff_mol.conformers[0]


Magnitude,[[0.5245067775886751 1.3887068569374834 0.04129898842136696] [-0.12353766343584843 2.6007942887991673 0.6765865388552703] [0.8094026744770679 3.630954996323167 0.3961031431974716] [0.3759659598724559 4.916622034464965 0.8578772699516551] [1.164037671827099 5.761213422193882 0.6184670508978817] [-0.6880064399342991 4.9425134602936724 1.4026063134693987] [-0.13253810738227711 0.09703388202890413 0.4539484653893632] [0.5064231589489996 -1.0778076717148664 -0.24600767359767173] [2.003215236622459 -1.1117033145328097 -0.022810522923824714] [2.3573298172863275 -2.4124822164639808 -0.46099165533349745] [3.7695975581570256 -2.656490388305661 -0.4062617415541381] [4.043937829583948 -3.743842323185933 -0.772952917116957] [4.439459370307802 -1.7505710390229836 -0.007173606771986949] [-1.626039842588408 0.15867357694536707 0.23623055752820063] [-2.350931129166617 -1.1241461369992984 0.5685158144885729] [-2.2149660918409886 -1.9663017224948316 -0.5700994384279339] [-3.014663186877294 -3.1624763485298866 -0.49774046252701126] [-2.90894448286594 -3.816435919289861 -1.4743868480103632] [-3.638266919713591 -3.3151778939944676 0.5065851389435078] [1.5640136953119885 1.3435103559642434 0.3503603612802047] [0.4777463788335295 1.4357968999338873 -1.0427088180786162] [-1.091377582994527 2.834586698244899 0.23895778225667738] [-0.2386821752402214 2.475952550997161 1.7545549412770232] [0.026240696172360193 -0.01969306214239789 1.4555429635620356] [0.054679776208696025 -1.977454343375275 0.15370745247397125] [0.2565149238750474 -0.9952376465170408 -1.2978236532534688] [2.52882795650238 -0.35755666345098597 -0.6038253841854971] [2.251859440941485 -0.9853589329241575 1.0328283245115129] [-1.7780412443364255 0.43747795766145964 -0.8026472364842862] [-2.0056363916363846 0.9488186044403423 0.8751710923771049] [-1.9438195918009054 -1.6001784207817558 1.4624100328272387] [-3.39830807799539 -0.8997415415023987 0.7504448445302194]]
Units,angstrom


In [40]:
retrieved.esp_settings

ESPSettings(basis='def2-tzvpp', method='wb97x-d', grid_settings=LatticeGridSettings(type='fcc', spacing=0.5, inner_vdw_scale=1.4, outer_vdw_scale=2.0), pcm_settings=None, psi4_dft_grid_settings=<DFTGridSettings.Default: 'default'>, ddx_settings=None)

In [48]:
esp_settings = ESPSettings(grid_settings=LatticeGridSettings(type='fcc', spacing=0.5, inner_vdw_scale=1.4, outer_vdw_scale=2.0))


calculate_resp_charges(
    openff_mol=openff_mol,
    grid= retrieved.grid_coordinates,
    esp= retrieved.esp.reshape(-1,1),
    qc_data_settings= retrieved.esp_settings
)

[-0.29,
 0.1213,
 -0.3505,
 0.8191,
 -0.2951,
 -0.3605,
 -0.1936,
 -0.29,
 0.1213,
 -0.3505,
 0.8191,
 -0.2951,
 -0.3605,
 -0.29,
 0.1213,
 -0.3505,
 0.8191,
 -0.2951,
 -0.3605,
 0.221,
 0.221,
 0.0932,
 0.0932,
 0.3761,
 0.221,
 0.221,
 0.0932,
 0.0932,
 0.221,
 0.221,
 0.0932,
 0.0932]